In [2]:
import pandas as pd
import yaml
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

#### Race YAML structure
> Update to match data for each individual race

In [2]:
# utmb_race = {
#     'race_name': 'Ultra Trail du Mont Blanc',
#     'race_loc': 'Chamonix, France',
#     'race_dist': '100M',
#     'race_uris': {
#         2017: '142.utmb-utmb-',
#         2018: '142.utmb-utmb-',
#         2019: '142.utmb-utmb-',
#         2021: '142.utmb-utmb-',
#         2022: '142.utmb-montblancutmb-',
#         2023: '142.daciautmb-montblancutmb',
#         2024: '142.hokautmbmont-blancutmb',
#         2025: '142.hokautmbmont-blancutmb'
        
#     },
#     'dates': {
#         2017: 'August 31, 2017',
#         2018: 'August 31, 2018',
#         2019: 'August 30, 2019',
#         2021: 'August 27, 2021',
#         2022: 'August 26, 2022',
#         2023: 'September 1, 2023',
#         2024: 'August 30, 2024',
#         2025: 'August 29, 2025'
#     }
# }

# with open("../config/utmb_100m.yaml", "w") as f:
#     yaml.dump(utmb_race, f)

#### Run Once 

In [17]:
def scrape_all_pages(driver, year, race_date, race_name, race_loc, race_dist):
    data = []
    page = 1

    while True:
        print(f"Scraping page {page} for {race_name} {year}")
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        rows = soup.find_all('div', class_='my-table_row__nlm_j')
        print(f"Total rows found on page {page}: {len(rows)}")

        # Debug: print all visible pagination links
        pagination_links = driver.find_elements(By.XPATH, "//a[contains(@class, 'pagination_paginate_link')]")
        print("Found page links:", [link.text for link in pagination_links])

        for row in rows:
            cells = row.find_all('div', class_='my-table_cell__z__zN')
            if len(cells) < 6:
                continue

            rank = cells[0].get_text(strip=True)
            status = 'DNF' if rank.upper() == 'DNF' else 'Finisher'
            name = cells[1].get_text(strip=True).lower()
            nationality_raw = cells[2].get_text(strip=True).lower()
            nationality = 'USA' if 'united states of america' in nationality_raw else nationality_raw.upper()
            gender_raw = cells[3].get_text(strip=True)
            gender = 'M' if 'Men' in gender_raw else 'F' if 'Women' in gender_raw else gender_raw
            age_category = cells[4].get_text(strip=True)
            time_str = cells[5].get_text(strip=True)

            data.append({
                'Date': race_date,
                'Year': int(year),
                'Rank': rank,
                'Status': status,
                'Name': name,
                'Nationality': nationality,
                'Gender': gender,
                'Age_Category': age_category,
                'Time': time_str,
                'Race': race_name,
                'Race_Loc': race_loc,
                'Race_Dist': race_dist
            })

        # Try to find the next page button by its text label
        try:
            next_page = str(page + 1)
            next_link = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, f"//a[normalize-space(text())='{next_page}']"))
            )
            driver.execute_script("arguments[0].click();", next_link)
            page += 1
            time.sleep(3)  # Delay to avoid rate limits
        except:
            print("No more pages found.")
            break

    return pd.DataFrame(data)


def scrape_utmb_race(year, race_date, race_name, race_loc, race_dist, race_uri):
    url = f"https://montblanc.utmb.world/results?year={year}&raceUri={race_uri}.{year}"
    
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    df = scrape_all_pages(driver, year, race_date, race_name, race_loc, race_dist)
    driver.quit()
    return df

## Insert race yaml for desired query and run the next two cell blocks
#### Update "../config/{race.yaml}"

In [18]:
race_yaml = "../config/mont_blanc_utmb_yaml/festival_des_templiers_100k.yaml"

with open(race_yaml, "r") as f:
    race = yaml.safe_load(f)

In [20]:
all_results = []

for year, date in race["dates"].items():
    uri = race["race_uris"].get(year)
    if not uri:
        print(f"No URI found for {year}, skipping.")
        continue

    df = scrape_utmb_race(
        year=year,
        race_date=date,
        race_name=race["race_name"],
        race_loc=race["race_loc"],
        race_dist=race["race_dist"],
        race_uri=uri
    )
    all_results.append(df)

# Combine all results into a single DataFrame
race_df = pd.concat(all_results, ignore_index=True)

Scraping page 1 for Festival des Templiers 100K 2021
Total rows found on page 1: 50
Found page links: ['', '1', '2', '...', '15', '16', '']
Scraping page 2 for Festival des Templiers 100K 2021
Total rows found on page 2: 50
Found page links: ['', '1', '2', '3', '...', '15', '16', '']
Scraping page 3 for Festival des Templiers 100K 2021
Total rows found on page 3: 50
Found page links: ['', '1', '2', '3', '4', '...', '15', '16', '']
Scraping page 4 for Festival des Templiers 100K 2021
Total rows found on page 4: 50
Found page links: ['', '1', '2', '3', '4', '5', '...', '15', '16', '']
Scraping page 5 for Festival des Templiers 100K 2021
Total rows found on page 5: 50
Found page links: ['', '1', '2', '3', '4', '5', '6', '...', '15', '16', '']
Scraping page 6 for Festival des Templiers 100K 2021
Total rows found on page 6: 50
Found page links: ['', '1', '2', '...', '5', '6', '7', '...', '15', '16', '']
Scraping page 7 for Festival des Templiers 100K 2021
Total rows found on page 7: 50
Foun

In [27]:
# Sanity Check. Compare values to Ultrasignup results.
status_counts = race_df.groupby(['Year', 'Status']).size().unstack(fill_value=0)
status_counts

Status,DNF,Finisher
Year,,
2021,0,769
2022,299,809
2023,195,978
2024,157,1068
2025,208,1282


In [28]:
race_df

,Date,Year,Rank,Status,Name,Nationality,Gender,Age_Category,Time,Race,Race_Loc,Race_Dist
0,2021-10-22,2021,1,Finisher,vincent viet,FRANCE,M,35-39,10:52:25,Festival des Templiers 100K,"Millau, France",100K
1,2021-10-22,2021,2,Finisher,aurelien collet,FRANCE,M,40-44,11:03:54,Festival des Templiers 100K,"Millau, France",100K
2,2021-10-22,2021,3,Finisher,clement desille,FRANCE,M,20-34,11:28:34,Festival des Templiers 100K,"Millau, France",100K
3,2021-10-22,2021,4,Finisher,matthieu durand,FRANCE,M,20-34,11:30:12,Festival des Templiers 100K,"Millau, France",100K
4,2021-10-22,2021,5,Finisher,romain olivier,FRANCE,M,35-39,11:30:18,Festival des Templiers 100K,"Millau, France",100K
...,...,...,...,...,...,...,...,...,...,...,...,...
5760,2025-10-17,2025,DNF,DNF,antoine henault,FRANCE,M,35-39,-,Festival des Templiers 100K,"Millau, France",100K
5761,2025-10-17,2025,DNF,DNF,fabien girard,FRANCE,M,40-44,-,Festival des Templiers 100K,"Millau, France",100K
5762,2025-10-17,2025,DNF,DNF,nicolas bonte,FRANCE,M,20-34,-,Festival des Templiers 100K,"Millau, France",100K
5763,2025-10-17,2025,DNF,DNF,tibolla laury,FRANCE,F,20-34,-,Festival des Templiers 100K,"Millau, France",100K


In [29]:
festival_des_templiers_100k_df = race_df

In [31]:
festival_des_templiers_100k_df.to_csv('../data/raw/utmb_format/festival_des_templiers_100K_df_raw.csv', index = False, encoding = 'utf-8')

# Some races such as Doi Ithanon use an a different url structure. The function below has been updated to scrape results from the updated format

In [17]:
race_yaml = "../../config/utmb_world_utmb_index_yaml/lake_sonoma_50m.yaml"

with open(race_yaml, "r") as f:
    race = yaml.safe_load(f)

In [18]:
def scrape_utmb_index_race(year, race_date, race_name, race_loc, race_dist, race_id_slug, race_id, series_id):
    url = f"https://utmb.world/utmb-index/races/{race_id_slug}.{year}?page=1"
    
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    data = []
    page = 1

    while True:
        print(f"Scraping page {page} for {race_name} {year}")
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        rows = soup.find_all('div', class_='my-table_row__nlm_j')
        print(f"Total rows found on page {page}: {len(rows)}")

         # Debug: print all visible pagination links
        pagination_links = driver.find_elements(By.XPATH, "//a[contains(@class, 'pagination_paginate_link')]")
        print("Found page links:", [link.text for link in pagination_links])

        for row in rows:
            cells = row.find_all('div', class_='my-table_cell__z__zN')
            if len(cells) < 6:
                continue

            rank = cells[0].get_text(strip=True)
            time_str = cells[1].get_text(strip=True)
            name_tag = cells[2].find('a')
            name = name_tag.get_text(strip=True).lower() if name_tag else ''
            nationality = cells[3].get_text(strip=True).split()[-1].upper()
            gender_raw = cells[4].get_text(strip=True)
            gender = 'M' if 'Men' in gender_raw else 'F' if 'Women' in gender_raw else gender_raw
            age_category = cells[5].get_text(strip=True)
            status = 'DNF' if rank.upper() == 'DNF' else 'Finisher'

            data.append({
                'Series_ID': series_id,
                'Race_ID': race_id,
                'Race_Date': race_date,
                'Year': int(year),
                'Rank': rank,
                'Status': status,
                'Name': name,
                'Nationality': nationality,
                'Gender': gender,
                'Age_Category': age_category,
                'Time': time_str,
                'Race_Name': race_name,
                'Race_Loc': race_loc,
                'Race_Dist': race_dist,
                'Race_ID': race_id
            })

        # Try to find the next page number
        try:
            next_page = str(page + 1)
            next_link = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, f"//a[normalize-space(text())='{next_page}']"))
            )
            driver.execute_script("arguments[0].click();", next_link)
            page += 1
            time.sleep(3)
        except:
            print("No more pages found.")
            break

    driver.quit()
    return pd.DataFrame(data)

In [19]:
all_results = []

for year, race_info in race["races"].items():
    date = race["dates"].get(year)
    if not date:
        print(f"No date found for {year}, skipping.")
        continue

    race_id = race_info.get("race_id")
    series_id = race.get("series_id")
    uri = race_info.get("race_uri")
    if not uri or not race_id:
        print(f"Missing race_id or URI for {year}, skipping.")
        continue

    df = scrape_utmb_index_race(
        year=year,
        race_date=date,
        race_name=race["race_name"],
        race_loc=race["race_loc"],
        race_dist=race["race_dist"],
        race_id_slug=uri,
        race_id=race_id,
        series_id=series_id
    )
    all_results.append(df)

race_index_df = pd.concat(all_results, ignore_index=True)

Scraping page 1 for Lake Sonoma 50 2016
Total rows found on page 1: 25
Found page links: ['', '1', '2', '...', '12', '13', '']
Scraping page 2 for Lake Sonoma 50 2016
Total rows found on page 2: 25
Found page links: ['', '1', '2', '3', '...', '12', '13', '']
Scraping page 3 for Lake Sonoma 50 2016
Total rows found on page 3: 25
Found page links: ['', '1', '2', '3', '4', '...', '12', '13', '']
Scraping page 4 for Lake Sonoma 50 2016
Total rows found on page 4: 25
Found page links: ['', '1', '2', '3', '4', '5', '...', '12', '13', '']
Scraping page 5 for Lake Sonoma 50 2016
Total rows found on page 5: 25
Found page links: ['', '1', '2', '3', '4', '5', '6', '...', '12', '13', '']
Scraping page 6 for Lake Sonoma 50 2016
Total rows found on page 6: 25
Found page links: ['', '1', '2', '...', '5', '6', '7', '...', '12', '13', '']
Scraping page 7 for Lake Sonoma 50 2016
Total rows found on page 7: 25
Found page links: ['', '1', '2', '...', '6', '7', '8', '...', '12', '13', '']
Scraping page 8 f

In [20]:
status_counts = race_index_df.groupby(['Year', 'Status']).size().unstack(fill_value=0)
status_counts

Status,DNF,Finisher
Year,,
2016,0,303
2017,51,300
2018,0,292
2019,65,280


In [21]:
race_index_df.head()

,Series_ID,Race_ID,Date,Year,Rank,Status,Name,Nationality,Gender,Age_Category,Time,Race,Race_Loc,Race_Dist
0,50487,27160,4-9-2016,2016,1,Finisher,jim walmsley,AMERICA,M,20-34,06:00:52,Lake Sonoma 50,"Healdsburg, CA",50M
1,50487,27160,4-9-2016,2016,2,Finisher,tim freriks,AMERICA,M,20-34,06:17:58,Lake Sonoma 50,"Healdsburg, CA",50M
2,50487,27160,4-9-2016,2016,3,Finisher,mario mendoza jr,AMERICA,M,20-34,06:30:44,Lake Sonoma 50,"Healdsburg, CA",50M
3,50487,27160,4-9-2016,2016,4,Finisher,dylan bowman,AMERICA,M,20-34,06:31:00,Lake Sonoma 50,"Healdsburg, CA",50M
4,50487,27160,4-9-2016,2016,5,Finisher,daniel metzger,AMERICA,M,20-34,06:37:23,Lake Sonoma 50,"Healdsburg, CA",50M


In [22]:
unique_nation = race_index_df['Nationality'].dropna().unique().tolist()
print(unique_nation)

['AMERICA', 'FRANCE', 'JAPAN', 'PERU', 'CANADA', 'SPAIN', 'GERMANY', 'INDIA', 'MEXICO', 'TAIPEI', 'AUSTRIA', 'IRELAND', 'IRAN', 'SWITZERLAND', 'BRAZIL', 'KINGDOM', 'AUSTRALIA', 'SWEDEN', 'SLOVAKIA', 'CHINA']


In [23]:
country_to_code = {
    'AMERICA': 'USA',
    'USA': 'USA',
    'POLAND': 'POL',
    'FRANCE': 'FRA',
    'UNITED KINGDOM': 'GBR',
    'ICELAND': 'ISL',
    'SPAIN': 'ESP',
    'SWITZERLAND': 'CHE',
    'IRELAND': 'IRL',
    'ITALY': 'ITA',
    'LATVIA': 'LVA',
    'SOUTH AFRICA': 'ZAF',
    'MOROCCO': 'MAR',
    'LUXEMBOURG': 'LUX',
    'CHINA': 'CHN',
    'VENEZUELA': 'VEN',
    'JAPAN': 'JPN',
    'PORTUGAL': 'PRT',
    'MAURITIUS': 'MUS',
    'NEW ZEALAND': 'NZL',
    'SWEDEN': 'SWE',
    'ARGENTINA': 'ARG',
    'SLOVAKIA': 'SVK',
    'GERMANY': 'DEU',
    'ALGERIA': 'DZA',
    'AUSTRALIA': 'AUS',
    'CAMBODIA': 'KHM',
    'NORWAY': 'NOR',
    'CROATIA': 'HRV',
    'MEXICO': 'MEX',
    'COLOMBIA': 'COL',
    'CANADA': 'CAN',
    'NETHERLANDS': 'NLD',
    'BRAZIL': 'BRA',
    'TÜRKIYE': 'TUR',
    'BELGIUM': 'BEL',
    'CHILE': 'CHL',
    'SOUTH KOREA': 'KOR',
    'DENMARK': 'DNK',
    'RUSSIA': 'RUS',
    'SLOVENIA': 'SVN',
    'UKRAINE': 'UKR',
    'SERBIA': 'SRB',
    'HUNGARY': 'HUN',
    'ROMANIA': 'ROU',
    'URUGUAY': 'URY',
    'ANDORRA': 'AND',
    'ECUADOR': 'ECU',
    'FINLAND': 'FIN',
    'HONG KONG, CHINA': 'HKG',
    'BELARUS': 'BLR',
    'SINGAPORE': 'SGP',
    'CZECH REPUBLIC': 'CZE',
    'BULGARIA': 'BGR',
    'PERU': 'PER',
    'GREECE': 'GRC',
    'PHILIPPINES': 'PHL',
    'LITHUANIA': 'LTU',
    'MALTA': 'MLT',
    'BOSNIA AND HERZEGOVINA': 'BIH',
    'THAILAND': 'THA',
    'AUSTRIA': 'AUT',
    'ESTONIA': 'EST',
    'CHINESE TAIPEI': 'TWN',
    'INDONESIA': 'IDN',
    'VIETNAM': 'VNM',
    'INDIA': 'IND',
    'MALAYSIA': 'MYS',
    'COSTA RICA': 'CRI',
    'ISRAEL': 'ISR',
    'SAUDI ARABIA': 'SAU',
    'NIGER': 'NER',
    'MOLDOVA': 'MDA',
    'EGYPT': 'EGY',
    'MONTENEGRO': 'MNE',
    'KOSOVO': 'XKX',
    'IRAN': 'IRN',
    'EL SALVADOR': 'SLV',
    'LEBANON': 'LBN',
    'PARAGUAY': 'PRY',
    'PUERTO RICO': 'PRI',
    'OMAN': 'OMN',
    'ALBANIA': 'ALB',
    'BENIN': 'BEN',
    'NORTH MACEDONIA (REPUBLIC OF)': 'MKD',
    'TUNISIA': 'TUN',
    'NEPAL': 'NPL',
    'QATAR': 'QAT',
    'DOMINICAN REPUBLIC': 'DOM',
    'SAN MARINO': 'SMR',
    'NICARAGUA': 'NIC',
    'GUATEMALA': 'GTM',
    'GAMBIA': 'GMB',
    'MONACO': 'MCO',
    'KYRGYZSTAN': 'KGZ',
    'KENYA': 'KEN',
    'BOLIVIA': 'BOL',
    'BAHRAIN': 'BHR',
    'MACAO, CHINA': 'MAC',
    'TOGO': 'TGO',
    'CYPRUS': 'CYP',
    'KUWAIT': 'KWT',
    'NAMIBIA': 'NAM',
    'ZIMBABWE': 'ZWE',
    'UGANDA': 'UGA',
    'LIECHTENSTEIN': 'LIE',
    'HAITI': 'HTI',
    'SWAZILAND': 'SWZ',
    'MONGOLIA': 'MNG',
    'CAMEROON': 'CMR',
    'KAZAKHSTAN': 'KAZ',
    'NEUTRAL ATHLETE': None,
    'UNITED ARAB EMIRATES': 'ARE',
    'MADAGASCAR': 'MDG',
    'AFGHANISTAN': 'AFG',
    'HONDURAS': 'HND',
    'GEORGIA': 'GEO',
    'JORDAN': 'JOR',
    'AZERBAIJAN': 'AZE',
    'BANGLADESH': 'BGD',
    'SENEGAL': 'SEN',
    'BRUNEI': 'BRN',
    'KINGDOM': 'GBR',
    'ZEALAND': None,
    'KOREA': None,
    'TAIPEI': 'TPE',
    'AFRICA': None,
    'ATHLETE': None,
    'SAO TOME AND PRINCIPE': 'STP',
    'PANAMA': 'PAN',
    'CONGO': 'COG',
    'JAMAICA': 'JAM',
    'FRENCH POLYNESIA': 'PYF',
    'IRAQ': 'IRQ',
    'UZBEKISTAN': 'UZB',
    'PAKISTAN': 'PAK',
    'NORTHERN MARIANA ISLANDS': 'MNP',
    'SOUTH SUDAN': 'SSD',
    'ARMENIA': 'ARM',
    'BOTSWANA': 'BWA',
    'BAHAMAS': 'BHS',
    'NORTH KOREA': 'PRK',
    'VANUATU': 'VUT'
}

In [24]:
race_index_df['Nationality'] = race_index_df['Nationality'].map(country_to_code)

In [25]:
race_index_df.head()

,Series_ID,Race_ID,Date,Year,Rank,Status,Name,Nationality,Gender,Age_Category,Time,Race,Race_Loc,Race_Dist
0,50487,27160,4-9-2016,2016,1,Finisher,jim walmsley,USA,M,20-34,06:00:52,Lake Sonoma 50,"Healdsburg, CA",50M
1,50487,27160,4-9-2016,2016,2,Finisher,tim freriks,USA,M,20-34,06:17:58,Lake Sonoma 50,"Healdsburg, CA",50M
2,50487,27160,4-9-2016,2016,3,Finisher,mario mendoza jr,USA,M,20-34,06:30:44,Lake Sonoma 50,"Healdsburg, CA",50M
3,50487,27160,4-9-2016,2016,4,Finisher,dylan bowman,USA,M,20-34,06:31:00,Lake Sonoma 50,"Healdsburg, CA",50M
4,50487,27160,4-9-2016,2016,5,Finisher,daniel metzger,USA,M,20-34,06:37:23,Lake Sonoma 50,"Healdsburg, CA",50M


In [26]:
# Keep original
original_df = race_index_df.copy()

# Filter finishers
finishers_df = original_df[original_df['Status'].str.upper() != 'DNF'].copy()

# Sort and rank
finishers_df = finishers_df.sort_values(by=['Race_ID', 'Gender', 'Time'])
finishers_df['Gender_Rank'] = finishers_df.groupby(['Race_ID', 'Gender']).cumcount() + 1

# Merge ranks back into original
race_index_df = original_df.merge(
    finishers_df[['Race_ID', 'Name', 'Gender', 'Time', 'Gender_Rank']],
    on=['Race_ID', 'Name', 'Gender', 'Time'],
    how='left'
)


# Replace NaN with blank
race_index_df['Gender_Rank'] = race_index_df['Gender_Rank'].fillna('')

In [29]:
# Current columns
cols = list(race_index_df.columns)

# Desired order: move Gender_Rank after Gender
cols.remove('Gender_Rank')
gender_index = cols.index('Gender') + 1
cols.insert(gender_index, 'Gender_Rank')

# Apply new order
race_index_df = race_index_df[cols]

In [30]:
race_index_df.head(15)

,Series_ID,Race_ID,Date,Year,Rank,Status,Name,Nationality,Gender,Gender_Rank,Age_Category,Time,Race,Race_Loc,Race_Dist
0,50487,27160,4-9-2016,2016,1,Finisher,jim walmsley,USA,M,1.0,20-34,06:00:52,Lake Sonoma 50,"Healdsburg, CA",50M
1,50487,27160,4-9-2016,2016,2,Finisher,tim freriks,USA,M,2.0,20-34,06:17:58,Lake Sonoma 50,"Healdsburg, CA",50M
2,50487,27160,4-9-2016,2016,3,Finisher,mario mendoza jr,USA,M,3.0,20-34,06:30:44,Lake Sonoma 50,"Healdsburg, CA",50M
3,50487,27160,4-9-2016,2016,4,Finisher,dylan bowman,USA,M,4.0,20-34,06:31:00,Lake Sonoma 50,"Healdsburg, CA",50M
4,50487,27160,4-9-2016,2016,5,Finisher,daniel metzger,USA,M,5.0,20-34,06:37:23,Lake Sonoma 50,"Healdsburg, CA",50M
5,50487,27160,4-9-2016,2016,6,Finisher,matt flaherty,USA,M,6.0,20-34,06:53:34,Lake Sonoma 50,"Healdsburg, CA",50M
6,50487,27160,4-9-2016,2016,7,Finisher,ben koss,USA,M,7.0,35-39,07:01:14,Lake Sonoma 50,"Healdsburg, CA",50M
7,50487,27160,4-9-2016,2016,8,Finisher,karl meltzer,USA,M,8.0,45-49,07:02:49,Lake Sonoma 50,"Healdsburg, CA",50M
8,50487,27160,4-9-2016,2016,9,Finisher,jeremy wolf,USA,M,9.0,35-39,07:05:55,Lake Sonoma 50,"Healdsburg, CA",50M
9,50487,27160,4-9-2016,2016,10,Finisher,zach bitter,USA,M,10.0,20-34,07:09:08,Lake Sonoma 50,"Healdsburg, CA",50M


In [31]:
race_index_df.to_csv('../../data/raw_data/utmb_format/lake_sonoma_50m_clean.csv', index = False, encoding = 'utf-8')